In [615]:
# -*- coding: utf8 -*-
from os.path import expanduser
home = expanduser("~")
repertoire=home+"/ownCloud/Cours/Bordeaux/L1-LinguistiqueGenerale/Kalaba-Project/16-K1"
repertoire="/Users/gilles/Dropbox/Partage-Broadway/GAM/Caches/NosCaches/Kalabas/K300"
serie=repertoire+"/"
#########################IMPORTS############################################
import codecs, optparse
import re, random
import sys,os,time
import string
import yaml
import ParFuMor as PFM
from ParFuMor import *
import pickle

In [616]:
debug=0
print_glose=True
print_cloze=True
print_radicaux=False
cloze_expanded=True
numeros={'1':'Un','2':'Deux','3':'Trois','4':'Quatre','5':'Cinq'}
personnes={'1sg':'UnSg','2sg':'DeuxSg','3sg':'TroisSg','1du':'UnDu','2du':'DeuxDu','3du':'TroisDu','1pl':'UnPl','2pl':'DeuxPl','3pl':'TroisPl'}

commandGrapho="\\newcommand{\\%s}{\\strutgb{0pt}\\grapho{%s}}"
#commandGrapho="\\newcommand{\\%s}{\\strutgb{0pt}{\\dn %s}}"
#commandGrapho=u"\\newcommand{\\%s}{\\strutgb{0pt}{%s}}"
commandPhono=u"\\newcommand{\\%sP}{\\textipa{%s}}"
commandGlose=u"\\newcommand{\\%sG}{\\textGlose{%s}}"

In [617]:
with open(serie+"Phonology.yaml", 'r') as stream:
    phonology=yaml.load(stream)
with open(serie+"Stems.yaml", 'r') as stream:
    stems=yaml.load(stream)
with open(serie+"MorphoSyntax.yaml", 'r') as stream:
    morphosyntax=yaml.load(stream)
with open(serie+"Hierarchie-S2.pkl", 'rb') as input:
   PFM.hierarchieCF = pickle.load(input)
with open(serie+"Lexique-S2.pkl", 'rb') as input:
   PFM.lexique = pickle.load(input)
with open(serie+"Regles-S2.pkl", 'rb') as input:
   PFM.regles = pickle.load(input)

In [618]:
PFM.lexique.lexemes[u'dans']

case PREP:	Tak	dans	Tak	\textRadical{Tak}	radical: Tak	racine:  


Tak, PREP, DANS
		Tak :
			PREP:	Tak	dans	Tak	\textRadical{Tak}	radical: Tak	racine: 

#### Définition des segments

In [619]:
consonnes=phonology["consonnes"]
voyelles=phonology["voyelles"]
gabarits=phonology["gabarits"]
derives=phonology["derives"]
nom_classe=phonology["nom_classe"]
nom_apo=phonology["apophonies"]
nom_mut=phonology["mutations"]
syllabes=phonology["syllabes"]

#### Attention aux correspondances pour les syllabes
YAML interprète la clé no comme False

In [620]:
def parse_grapho(graphie):
    def change_monograms(chaine):
        result=""
        for monogramme in chaine:
            if monogramme in monogrammes:
                result+=monogrammes[monogramme]
            else:
                result+=monogramme
        return result

    def change_bigrams(chaine):
        result=change_monograms(chaine)
        for bigramme in bigrammes:
            result=result.replace(bigramme,bigrammes[bigramme])
        return result

    
    chunks=re.findall(ur"\s+|[ptkqbdgmnNfTsSvDzZjwRrlLyvWJ]?[aeiouAEIUO]?", graphie,re.U)
    result=[]
    if debug or "m" in graphie: print graphie
    for chunk in chunks:
        if debug or "m" in graphie: print chunk,
        #ATTENTION : YAML interprète "no" comme FALSE
        if chunk in syllabes.keys():
            result.append(syllabes[chunk])
        else:
            result.append(chunk)
    if debug or "m" in graphie: 
        print result
        print
    return change_bigrams("".join(result))

In [621]:
parse_grapho(u"SugiNabUg")

u'.sugihabUg'

In [622]:
def parse_cloze(glose):
    if debug: print [glose]
    chunks=re.findall(r"\\cacherGloses{([^}]*)?}|(\w+)", glose,re.UNICODE)
    result=[]
    for chunk in chunks:
        result.extend([x for x in chunk if x!=""])
    return (u"%s;"%len(result)+";".join(result),"".join(result))

In [623]:
if "Cas" in morphosyntax:
    casSyntagmes=morphosyntax["Cas"]
else:
    casSyntagmes=""
lexiquePrepositions=[stems["PREP"][x][0] for x in stems["PREP"]]
casPreposition={}
for preposition in lexiquePrepositions:
    prep=preposition.upper()
    if casSyntagmes and prep in casSyntagmes:
        casPreposition[preposition]=casSyntagmes[prep]
    elif casSyntagmes and "PREP" in casSyntagmes:
        casPreposition[preposition]=casSyntagmes["PREP"]
    else:
        casPreposition[preposition]=""
glosePREP={x:u"%s%s"%(x.upper(),cacherGloses("["+casPreposition[x].strip("+").capitalize()+"]")) for x in casPreposition if "+" in casPreposition[x]}

In [624]:
glosePREP

{'ENTRE': u'ENTRE\\cacherGloses{[Obl]}',
 u'apr\xe8s': u'APR\xc8S\\cacherGloses{[Obl]}',
 'avec': u'AVEC\\cacherGloses{[Obl]}',
 'dans': u'DANS\\cacherGloses{[Obl]}',
 'de': u'DE\\cacherGloses{[Obl]}',
 u'derri\xe8re': u'DERRI\xc8RE\\cacherGloses{[Obl]}',
 'devant': u'DEVANT\\cacherGloses{[Obl]}',
 'pour': u'POUR\\cacherGloses{[Dat]}',
 'sous': u'SOUS\\cacherGloses{[Obl]}',
 'sur': u'SUR\\cacherGloses{[Obl]}',
 'vers': u'VERS\\cacherGloses{[Obl]}',
 u'\xe0': u'\xc0\\cacherGloses{[Dat]}'}

In [625]:
try:
    __IPYTHON__ 
    ipython=True
except: 
    ipython=False

version=os.path.basename("__file__")
time_stamp='%s' % time.strftime("%y%m%d-%H%M")
print "%% version : "+version
print "%% traitement : "+time_stamp

if ipython or True:
#    lexeme_nom="lexemes.txt"
#    phrase_nom="phrases.txt"
    pass
else:
    parser=optparse.OptionParser()
    parser.add_option("-o", "--out", dest="outfile", action="store_true", help="write to FILE")
    parser.add_option("-c", "--cloze", dest="print_cloze", action="store_true", help="write a CLOZE FILE")
    parser.add_option("-l", "--lexicon", dest="print_lexique", action="store_true", help="append a lexicon")
    parser.add_option("-r", "--roots", dest="print_racines", action="store_true", help="append a root list")

    (options, args) = parser.parse_args()
    lexeme_nom=args[0]
    phrase_nom=args[1]

%% version : __file__
%% traitement : 190112-1931


In [626]:
def recoder(chaine,table,boolUTF8=True):
    if type(chaine)==str:
        temp=unicode(chaine.decode('utf8')).translate(table)
        result=temp
    elif type(chaine)==unicode:
        result=chaine.translate(table)
    if boolUTF8:
        return result
    else:
        return result.encode("utf8")
#translit=string.maketrans(u'iueoaftgzZvjkSpN',u'tgazpHTGZJVkXyxI')

In [627]:
phonoIn =  unicode(phonology["translations"]["grapho"]["in"])
graphoIn = [ord(char) for char in phonoIn]
graphoOut = unicode(phonology["translations"]["grapho"]["out"])
translit = dict(zip(graphoIn, graphoOut))
if "monogrammes" in phonology["translations"]:
    monogrammes = phonology["translations"]["monogrammes"]
else:
    monogrammes = {}
bigrammes = phonology["translations"]["bigrammes"]


In [628]:
accentedIn = unicode(phonology["translations"]["deaccent"]["in"])
deaccentIn = [ord(char) for char in accentedIn]
deaccentOut = unicode(phonology["translations"]["deaccent"]["out"])
deaccent = dict(zip(deaccentIn, deaccentOut))

deligatures=phonology["translations"]["deligatures"]

In [629]:
#translit

In [630]:
tipaIn = unicode(phonology["translations"]["ipa"]["in"])
ipaIn = [ord(char) for char in tipaIn]
ipaOut = unicode(phonology["translations"]["ipa"]["out"])
toipa = dict(zip(ipaIn, ipaOut))

In [631]:
tableaux={}
gloseClozes={}
declarations=[]
for categorie in PFM.lexique.catLexeme:
    if not categorie in tableaux:
        tableaux[categorie]=set()
        gloseClozes[categorie]=[]
    if verbose: print categorie
    for lexeme in PFM.lexique.catLexeme[categorie]:
        if verbose: print PFM.lexique.lexemes[lexeme]
        for case in PFM.lexique.lexemes[lexeme].paradigme.cases:
            caseGlose=case.glose
            if categorie in PFM.categoriesMajeures:
                nom=PFM.lexique.lexemes[lexeme].nom
            else:
                nom=PFM.lexique.lexemes[lexeme].nom.upper()
                gloseCase=case.glose
                if categorie=="PREP" and gloseCase in glosePREP:
                    caseGlose=glosePREP[gloseCase]
                elif categorie=="PREP":
                    caseGlose=caseGlose.upper()
#                print caseGlose
            ref=PFM.modifierGlose(nom,case.sigma,"ref")
            ref=recoder(ref,deaccent)
            for ligature in deligatures:
                ref=ref.replace(ligature,deligatures[ligature])
            for pers in personnes:
                ref=ref.replace(pers,personnes[pers])
            for num in numeros:
                ref=ref.replace(num,numeros[num])
#            phono=case.forme
#            print ("lexeme",lexeme)
            grapho=chaine2utf8(re.sub(ur"\s+",ur"",parse_grapho(recoder(case.forme,translit))))
            if print_radicaux:
                phono=case.detoure.replace(" ","")
            else:
                phono=case.forme.replace(" ","")
#            print [ref,grapho]
            declarations.append(commandGrapho%(ref,grapho))
            declarations.append(commandPhono%(ref,phono))
            declarations.append(commandGlose%(ref,caseGlose))
            if print_glose:
                tableaux[categorie].add("\\"+ref+" & \\"+ref+"P & \\"+ref+"G \\\\")
            else:
                tableaux[categorie].add("\\"+ref+" & \\"+ref+"P & \\blanc{\\"+ref+"G} \\\\")
            if print_cloze:
                vedette=nom.split(".")[0]
                gloses,strGlose=parse_cloze(case.glose)
                phono=case.forme.replace(" ","")
                if debug: print [grapho, recoder(phono,toipa)]
                if cloze_expanded:
                    try:
                        cloze=u";".join([ref,vedette,
                                         categorie,
                                         recoder(case.radical,toipa),recoder(case.racine,toipa),
                                         recoder(phono,toipa),recoder(case.decoupe,toipa),
                                         case.sigma,
                                         strGlose,
                                         gloses])
                    except NameError:
                        cloze=u";".join([ref,vedette,
                                         categorie,
                                         recoder(case.radical,toipa),recoder(case.racine,toipa),
                                         recoder(phono,toipa),
                                         strGlose,
                                         gloses])
                else:
                    try:
                        cloze=u";".join([ref,vedette,categorie,recoder(phono,toipa),recoder(case.decoupe,toipa),case.sigma,gloses])
                    except NameError:
                        cloze=u";".join([ref,vedette,categorie,recoder(phono,toipa),gloses])
                gloseClozes[categorie].append(cloze)
    

SusamuZI
Su sa mu ZI  [u'Su', u'sa', u'mu', u'ZI', u'']

SusamuZU
Su sa mu ZU  [u'Su', u'sa', u'mu', u'ZU', u'']

SusamuZa
Su sa mu Za  [u'Su', u'sa', u'mu', u'Za', u'']

SusamuZu
Su sa mu Zu  [u'Su', u'sa', u'mu', u'Zu', u'']

SusamuZId
Su sa mu ZI d  [u'Su', u'sa', u'mu', u'ZI', u'd', u'']

SusamuZUd
Su sa mu ZU d  [u'Su', u'sa', u'mu', u'ZU', u'd', u'']

SusamuZad
Su sa mu Za d  [u'Su', u'sa', u'mu', u'Za', u'd', u'']

SusamuZud
Su sa mu Zu d  [u'Su', u'sa', u'mu', u'Zu', u'd', u'']

SusamuZIZ
Su sa mu ZI Z  [u'Su', u'sa', u'mu', u'ZI', u'Z', u'']

SusamuZUZ
Su sa mu ZU Z  [u'Su', u'sa', u'mu', u'ZU', u'Z', u'']

SusamuZaZ
Su sa mu Za Z  [u'Su', u'sa', u'mu', u'Za', u'Z', u'']

SusamuZuZ
Su sa mu Zu Z  [u'Su', u'sa', u'mu', u'Zu', u'Z', u'']

kUsamuZI
kU sa mu ZI  [u'kU', u'sa', u'mu', u'ZI', u'']

kUsamuZU
kU sa mu ZU  [u'kU', u'sa', u'mu', u'ZU', u'']

kUsamuZa
kU sa mu Za  [u'kU', u'sa', u'mu', u'Za', u'']

kUsamuZu
kU sa mu Zu  [u'kU', u'sa', u'mu', u'Zu', u'']

kUsamuZId
kU sa 


balmimUUDI
ba l mi mU U DI  [u'ba', u'l', u'mi', u'mU', u'U', u'DI', u'']

zUbablam
zU ba b la m  [u'zU', u'ba', u'b', u'la', u'm', u'']

bUbablam
bU ba b la m  [u'bU', u'ba', u'b', u'la', u'm', u'']

vabablam
va ba b la m  [u'va', u'ba', u'b', u'la', u'm', u'']

bablamI
ba b la mI  [u'ba', u'b', u'la', u'mI', u'']

tumaNuzU
tu ma Nu zU  [u'tu', u'ma', u'Nu', u'zU', u'']

bUtumaNuz
bU tu ma Nu z  [u'bU', u'tu', u'ma', u'Nu', u'z', u'']

vatumaNuz
va tu ma Nu z  [u'va', u'tu', u'ma', u'Nu', u'z', u'']

tumaNuzI
tu ma Nu zI  [u'tu', u'ma', u'Nu', u'zI', u'']

tumaNUDU
tu ma NU DU  [u'tu', u'ma', u'NU', u'DU', u'']

bUtumaNUD
bU tu ma NU D  [u'bU', u'tu', u'ma', u'NU', u'D', u'']

vatumaNUD
va tu ma NU D  [u'va', u'tu', u'ma', u'NU', u'D', u'']

tumaNUDI
tu ma NU DI  [u'tu', u'ma', u'NU', u'DI', u'']

zUtatmuN
zU ta t mu N  [u'zU', u'ta', u't', u'mu', u'N', u'']

bUtatmuN
bU ta t mu N  [u'bU', u'ta', u't', u'mu', u'N', u'']

vatatmuN
va ta t mu N  [u'va', u'ta', u't', u'mu', u'N', u'']



wU zu ga ya m  [u'wU', u'zu', u'ga', u'ya', u'm', u'']

zugayamU
zu ga ya mU  [u'zu', u'ga', u'ya', u'mU', u'']

Nizugayam
Ni zu ga ya m  [u'Ni', u'zu', u'ga', u'ya', u'm', u'']

zwigUzamU
z wi gU za mU  [u'z', u'wi', u'gU', u'za', u'mU', u'']

wUwigUzam
wU wi gU za m  [u'wU', u'wi', u'gU', u'za', u'm', u'']

wigUzamU
wi gU za mU  [u'wi', u'gU', u'za', u'mU', u'']

NiwigUzam
Ni wi gU za m  [u'Ni', u'wi', u'gU', u'za', u'm', u'']

zkiwiSaamU
z ki wi Sa a mU  [u'z', u'ki', u'wi', u'Sa', u'a', u'mU', u'']

wUkiwiSaam
wU ki wi Sa a m  [u'wU', u'ki', u'wi', u'Sa', u'a', u'm', u'']

kiwiSaamU
ki wi Sa a mU  [u'ki', u'wi', u'Sa', u'a', u'mU', u'']

NikiwiSaam
Ni ki wi Sa a m  [u'Ni', u'ki', u'wi', u'Sa', u'a', u'm', u'']

zduyUfiamU
z du yU fi a mU  [u'z', u'du', u'yU', u'fi', u'a', u'mU', u'']

wUduyUfiam
wU du yU fi a m  [u'wU', u'du', u'yU', u'fi', u'a', u'm', u'']

duyUfiamU
du yU fi a mU  [u'du', u'yU', u'fi', u'a', u'mU', u'']

NiduyUfiam
Ni du yU fi a m  [u'Ni', u'du', u'yU', u'fi', u'

U yI m  [u'U', u'yI', u'm', u'']

ayam
a ya m  [u'a', u'ya', u'm', u'']

ayum
a yu m  [u'a', u'yu', u'm', u'']

ayIm
a yI m  [u'a', u'yI', u'm', u'']

ayIm
a yI m  [u'a', u'yI', u'm', u'']

Ipam
I pa m  [u'I', u'pa', u'm', u'']

Ipum
I pu m  [u'I', u'pu', u'm', u'']

IpIm
I pI m  [u'I', u'pI', u'm', u'']

IpIm
I pI m  [u'I', u'pI', u'm', u'']

Upam
U pa m  [u'U', u'pa', u'm', u'']

Upum
U pu m  [u'U', u'pu', u'm', u'']

UpIm
U pI m  [u'U', u'pI', u'm', u'']

UpIm
U pI m  [u'U', u'pI', u'm', u'']

apam
a pa m  [u'a', u'pa', u'm', u'']

apum
a pu m  [u'a', u'pu', u'm', u'']

apIm
a pI m  [u'a', u'pI', u'm', u'']

apIm
a pI m  [u'a', u'pI', u'm', u'']

mabUwnI
ma bU w nI  [u'ma', u'bU', u'w', u'nI', u'']

mabbiwan
ma b bi wa n  [u'ma', u'b', u'bi', u'wa', u'n', u'']

mabUwnU
ma bU w nU  [u'ma', u'bU', u'w', u'nU', u'']

mabUwfI
ma bU w fI  [u'ma', u'bU', u'w', u'fI', u'']

mabbiwaf
ma b bi wa f  [u'ma', u'b', u'bi', u'wa', u'f', u'']

mabUwfU
ma bU w fU  [u'ma', u'bU', u'w', u'fU', u'']



In [632]:
if print_radicaux:
    nomDeclaration="Declarations-Radicaux.tex"
    nomTableaux="Tableaux-Radicaux.yaml"
else:
    nomDeclaration="Declarations.tex"
    nomTableaux="Tableaux.yaml"

In [633]:
with codecs.open(serie+nomDeclaration, 'wb', encoding='utf8') as output:
    for declaration in declarations:
#        print type(declaration),declaration
        output.write(declaration+"\n")

In [634]:
if print_cloze:
    with codecs.open(serie+"Clozes.txt", 'wb', encoding='utf8') as output:
        for categorie in gloseClozes:
            output.write("#\t"+categorie+"\n#\n#\n")
            for cloze in gloseClozes[categorie]:
                output.write(cloze+"\n")
            output.write("#\n#\n#\n")

In [635]:
yaml.safe_dump(tableaux, file(serie+nomTableaux, 'w'), encoding='utf-8', allow_unicode=True)

In [636]:
#gloseClozes

In [637]:
[l for l in declarations if u"Prep" in l]

[]

In [638]:
chaine="pumak"
print "parse", parse_grapho(chaine)
print "parse, recoder",recoder(parse_grapho(chaine),translit)
print "recoder", recoder(chaine,translit)
print "recoder, parse",parse_grapho(recoder(chaine,translit))


parse pumak
pu ma k  ['pu', 'ma', 'k', '']

pumak
parse, recoder pumak
pu ma k  ['pu', 'ma', 'k', '']

giNab
recoder giNab
recoder, parse gihab


In [639]:
DG=ur"\offrirVdPstTroisSgCP{}    \DEFSgBDatP{}   \LilouBSgDatP{}   \INDSgCAbsP{}   \sanglierCSgAbsP{}   \DEFSgDErgP{}   \trappeurDSgErgP{}"
print " ".join([m for m in DG.split(" ") if m!=""][::-1])

\trappeurDSgErgP{} \DEFSgDErgP{} \sanglierCSgAbsP{} \INDSgCAbsP{} \LilouBSgDatP{} \DEFSgBDatP{} \offrirVdPstTroisSgCP{}
